In [1]:
import asyncio
from InlineAgent.agent import InlineAgent
from InlineAgent.tools import MCPHttp
from IPython.display import clear_output
from IPython.display import display
import ipywidgets as widgets

![Bastion DNS](./static/bastion.png)

Copy the _McpEksStack.BastionInstancePublicDnsName_ CDK stack output to the cell below

In [2]:
# Replace with your EC2 instance IP or hostname
EC2_HOST = "ec2-35-85-57-47.us-west-2.compute.amazonaws.com"
MCP_PORT = 8000

In [3]:
async def setup_agent():
    # Configure connection to your Kubernetes MCP server
    mcp_client = await MCPHttp.create(
        url=f"http://{EC2_HOST}:{MCP_PORT}/sse",
        # You can add headers like authorization headers if needed
        headers={},
        timeout=10,  # Connection timeout
        sse_read_timeout=300  # Read timeout (5 minutes)
    )

    # Create the InlineAgent with the MCP client
    agent = InlineAgent(
        foundation_model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        instruction="""You are a Kubernetes cluster management assistant that helps users manage their EKS cluster.
You have access to various kubectl commands through an MCP server.
When users ask you about Kubernetes resources or want to perform actions, use the appropriate tools.
Always show the relevant information clearly and explain what you're doing.
""",
        agent_name="kubernetes-assistant",
        action_groups=[
            {
                "name": "KubernetesActions",
                "description": "Tools for managing Kubernetes clusters",
                "mcp_clients": [mcp_client]
            }
        ]
    )

    return agent